<h1>Skyrim Alchemy Optimization</h1>
<h4>Blake Rayvid - <a href=https://github.com/brayvid>https://github.com/brayvid</a></h4>

Make the most of the ingredients you have. Maximize total magnitude (essentially in-game value) with integer linear programming in scipy.


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import milp, Bounds, LinearConstraint

## Read in ingredients and recipes
Uses local files "ingredients_have.csv" and "recipes_can_make.csv"

I made my CSVs using this helpful spreadsheet:<a href="https://docs.google.com/spreadsheets/d/1010C6ltqv7apuBoNYuFIFSBZER4YI03Y54kIsoKs5RI/edit?usp=sharing"> https://docs.google.com/spreadsheets/d/1010C6ltqv7apuBoNYuFIFSBZER4YI03Y54kIsoKs5RI/edit?usp=sharing </a>

In [ ]:
# Ingredients we have with quantity on hand
ingredients = pd.read_csv('ingredients_have.csv');ingredients

,Ingredient,Quantity
0,Blisterwort,4
1,Blue Butterfly Wing,4
2,Blue Dartwing,1
3,Blue Mountain Flower,24
4,Bone Meal,5
5,Butterfly Wing,6
6,Canis Root,2
7,Creep Cluster,1
8,Deathbell,6
9,Dragons Tongue,5


In [ ]:
# Potions list with magnitude and ingredient names (1,2 + optional 3rd)
recipes = pd.read_csv('recipes_can_make.csv')
recipes = recipes[recipes['Magnitude'] > 0];
recipes.head(20)

,Magnitude,Type,Ingredient 1,Ingredient 2,Ingredient 3,Effects,Effect 1,Effect 2,Effect 3,Effect 4,Effect 5,MyPotionID,Can Make
0,159,Mixed,Blue Dartwing,Blue Mountain Flower,Glow Dust,3,Restore Health,Damage Magicka Regen,Resist Shock,NaN,NaN,3028,True
1,156,Mixed,Blue Dartwing,Blue Mountain Flower,Nightshade,2,Restore Health,Damage Magicka Regen,NaN,NaN,NaN,3037,True
2,156,Mixed,Blue Dartwing,Blue Mountain Flower,Spider Egg,2,Restore Health,Damage Magicka Regen,NaN,NaN,NaN,3045,True
3,156,Mixed,Blue Dartwing,Blue Mountain Flower,Spriggan Sap,2,Restore Health,Damage Magicka Regen,NaN,NaN,NaN,3046,True
4,113,Mixed,Blisterwort,Blue Butterfly Wing,Blue Mountain Flower,4,Damage Stamina,Restore Health,Fortify Conjuration,Damage Magicka Regen,NaN,2130,True
5,113,Mixed,Blue Butterfly Wing,Blue Mountain Flower,Rock Warbler Egg,4,Fortify Conjuration,Damage Magicka Regen,Restore Health,Damage Stamina,NaN,2680,True
6,112,Mixed,Frost Mirriam,Histcarp,Purple Mountain Flower,4,Damage Stamina Regen,Restore Stamina,Fortify Sneak,Resist Frost,NaN,10371,True
7,110,Mixed,Blue Butterfly Wing,Blue Mountain Flower,Butterfly Wing,3,Fortify Conjuration,Damage Magicka Regen,Restore Health,NaN,NaN,2666,True
8,110,Mixed,Blue Butterfly Wing,Blue Mountain Flower,Imp Stool,3,Fortify Conjuration,Damage Magicka Regen,Restore Health,NaN,NaN,2677,True
9,110,Mixed,Blue Butterfly Wing,Blue Mountain Flower,Swamp Fungal Pod,3,Fortify Conjuration,Damage Magicka Regen,Restore Health,NaN,NaN,2684,True


## Create recipe matrix A in Ax <= b
One row for each ingredient, one column for each potion. "1" indicates the ingredient is used in the potion.



In [ ]:
# Boolean matrix A says what ingredients are in what recipes
A = pd.DataFrame(0, index=range(len(ingredients)),columns=range(len(recipes)))
for i in range(len(recipes)):
  if ingredients.iloc[ingredients["Ingredient"].str.find(recipes.loc[i, "Ingredient 1"]).idxmax()]["Quantity"] > 0:
    A.iloc[ingredients["Ingredient"].str.find(recipes.loc[i, "Ingredient 1"]).idxmax(), i] = 1
  if ingredients.iloc[ingredients["Ingredient"].str.find(recipes.loc[i, "Ingredient 2"]).idxmax()]["Quantity"] > 0:
    A.iloc[ingredients["Ingredient"].str.find(recipes.loc[i, "Ingredient 2"]).idxmax(), i] = 1
  if not pd.isnull(recipes.loc[i, "Ingredient 3"]):
    if ingredients.iloc[ingredients["Ingredient"].str.find(recipes.loc[i, "Ingredient 3"]).idxmax()]["Quantity"] > 0:
      A.iloc[ingredients["Ingredient"].str.find(recipes.loc[i, "Ingredient 3"]).idxmax(), i] = 1
A.head(20)

,0,1,2,3,4,5,6,7,8,9,...,2375,2376,2377,2378,2379,2380,2381,2382,2383,2384
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## Set up optimization variables
find x to minimize f.x with Ax <= b, x >= lb

f = -1 * magnitude, b = qty of each ingredient on hand

In [ ]:
# Objective function f.x to minimize
f = np.array(-1 * recipes['Magnitude'],dtype=int); # f = -1*value so that minimizing f.x maximizes total value

In [ ]:
# Bounds
b_max = np.array(ingredients['Quantity'],dtype=int) # Cannot use more than we have on hand
x_lb = np.zeros(shape=len(recipes)) # Cannot use less than 0

In [ ]:
# milp parameters
bounds = Bounds(lb=x_lb)
constraint = LinearConstraint(A, ub=b_max)
integrality = np.ones(shape=len(recipes),dtype=int) # All x should be integers

# Perform optimization with scipy.optimize.milp

In [ ]:
# Perform optimization
res = milp(c=f, integrality=integrality, bounds=bounds, constraints=constraint)

## Display recommended potions to make

In [ ]:
# Display the potions we should make to maximize magnitude where the last column is quantity to make
total_magnitude = int(-res.fun)
num_potions = int(sum(res.x))
indices_to_make = np.nonzero(res.x > 0)
to_make_df = recipes.iloc[indices_to_make].copy()
to_make_df.loc[:,'QtyToMake'] = res.x[indices_to_make].astype(int)
to_make_df.head(len(to_make_df))

,Magnitude,Type,Ingredient 1,Ingredient 2,Ingredient 3,Effects,Effect 1,Effect 2,Effect 3,Effect 4,Effect 5,MyPotionID,Can Make,QtyToMake
6,112,Mixed,Frost Mirriam,Histcarp,Purple Mountain Flower,4,Damage Stamina Regen,Restore Stamina,Fortify Sneak,Resist Frost,NaN,10371,True,2
7,110,Mixed,Blue Butterfly Wing,Blue Mountain Flower,Butterfly Wing,3,Fortify Conjuration,Damage Magicka Regen,Restore Health,NaN,NaN,2666,True,4
11,109,Mixed,Blisterwort,Blue Mountain Flower,Spriggan Sap,3,Restore Health,Damage Magicka Regen,Fortify Smithing,NaN,NaN,2210,True,3
19,108,Mixed,Blisterwort,Blue Mountain Flower,Spider Egg,3,Restore Health,Damage Stamina,Damage Magicka Regen,NaN,NaN,2209,True,1
31,108,Mixed,Blue Mountain Flower,Bone Meal,Spider Egg,3,Fortify Conjuration,Damage Stamina,Damage Magicka Regen,NaN,NaN,3416,True,4
33,108,Mixed,Blue Mountain Flower,Glow Dust,Hagraven Feathers,3,Damage Magicka Regen,Damage Magicka,Fortify Conjuration,NaN,NaN,3628,True,1
34,108,Mixed,Blue Mountain Flower,Glow Dust,Swamp Fungal Pod,3,Damage Magicka Regen,Resist Shock,Restore Health,NaN,NaN,3643,True,1
35,108,Mixed,Blue Mountain Flower,Rock Warbler Egg,Spider Egg,3,Restore Health,Damage Stamina,Damage Magicka Regen,NaN,NaN,3780,True,1
45,107,Mixed,Creep Cluster,Ectoplasm,Skeever Tail,3,Restore Magicka,Damage Stamina Regen,Damage Health,NaN,NaN,6318,True,1
57,107,Mixed,Frost Mirriam,Purple Mountain Flower,Skeever Tail,3,Resist Frost,Fortify Sneak,Damage Stamina Regen,NaN,NaN,10519,True,1


In [ ]:
print(f"To maximize magnitude and therefore value, create {num_potions} potions of the {len(to_make_df)} unique types listed above for a total magnitude of {total_magnitude}.")

To maximize magnitude and therefore value, create 76 potions of the 42 unique types listed above for a total magnitude of 3905.
